In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
page_url = 'https://www.novibet.gr/en/sports'
driver.get(page_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'acceptCookies_button'))).click()

In [4]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[data-cy="closeBtn"]').click()

In [6]:
# Click Daily Coupon
dayly_coupon_button = driver.find_element(By.CSS_SELECTOR, 'a.ng-star-inserted[title="Daily coupon"]').click()

In [ ]:
# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract footbal text
football_string = daily_coupon_body.text

In [21]:
football_string = daily_coupon_text_football

In [95]:
football_string

"Daily coupon\nFootball\nBasketball\nTennis\n24 hours\n12 hours\n3 hours\nPopular First\nArgentina - Liga Profesional\nVelez Sarsfield\nCentral Cordoba Santiago\nWed 02:00\n1\n1.74\nX\n3.55\n2\n5.20\nO 2.5\n2.35\nU 2.5\n1.59\nGG\n2.25\nNG\n1.63\nAfrica - Africa Cup of Nations - Qualification\nBenin\nRwanda\nWed 17:00\n1\n1.74\nX\n3.45\n2\n4.85\nO 2.5\n2.25\nU 2.5\n1.63\nGG\n2.10\nNG\n1.68\nSierra Leone\nSao Tome and Principe\nWed 19:00\n1\n1.10\nX\n7.80\n2\n18.50\nO 3.5\n1.79\nU 3.5\n1.96\nMarkets are not available\nEngland - Isthmian Premier Division\nLewes\nCanvey Island\n21:45\n1\n2.15\nX\n3.45\n2\n2.95\nO 2.5\n1.87\nU 2.5\n1.83\nGG\n1.74\nNG\n1.97\nPotters Bar Town FC\nBognor Regis Town\n21:45\n1\n2.70\nX\n3.45\n2\n2.25\nO 2.5\n1.67\nU 2.5\n2.08\nGG\n1.57\nNG\n2.25\nEngland - League 1\nAccrington\nPlymouth\n21:45\n1\n5.20\nX\n3.95\n2\n1.65\nO 2.5\n1.74\nU 2.5\n2.10\nGG\n1.75\nNG\n2.05\nBarnsley\nSheffield Wed\n22:00\n1\n2.85\nX\n3.15\n2\n2.60\nO 2.5\n2.35\nU 2.5\n1.59\nGG\n2.00\nNG

In [106]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First',
                   'Markets are not available']

football_list = [x for x in initial_list if x not in remove_elements]

In [107]:
# Create sublists of based on Championship
championship = [x for x in football_list if ' - ' in x]
index_championship = [i for i,x in enumerate(football_list) if ' - ' in x]
sublists_championships = [football_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(football_list)])]

In [115]:
sublists_championships[5]

['England - National League North',
 'Banbury',
 'Curzon Ashton FC',
 '21:45',
 '1',
 '2.20',
 'X',
 '3.55',
 '2',
 '3.05',
 'O 2.5',
 '1.86',
 'U 2.5',
 '1.92',
 'GG',
 '1.71',
 'NG',
 '2.06',
 'Farsley Celtic FC',
 'Darlington',
 '21:45',
 '1',
 '2.85',
 'X',
 '3.60',
 '2',
 '2.25',
 'O 2.5',
 '1.79',
 'U 2.5',
 '1.99',
 'GG',
 '1.66',
 'NG',
 '2.15',
 'Kings Lynn Town',
 'AFC Fylde',
 '21:45',
 '1',
 '2.30',
 'X',
 '3.55',
 '2',
 '2.80',
 'O 2.5',
 '1.76',
 'U 2.5',
 '2.03',
 'GG',
 '1.63',
 'NG',
 '2.20',
 'Southport',
 'Alfreton Town FC',
 '21:45',
 '1',
 '2.30',
 'X',
 '3.45',
 '2',
 '2.95',
 'O 2.5',
 '1.95',
 'U 2.5',
 '1.83',
 'GG',
 '1.76',
 'NG',
 '1.99',
 'Spennymoor Town',
 'Leamington',
 '21:45',
 '1',
 '1.87',
 'X',
 '3.55',
 '2',
 '3.95',
 'O 2.5',
 '1.79',
 'U 2.5',
 '1.99',
 'GG',
 '1.70',
 'NG',
 '2.08']

In [116]:
teams1 = []
teams2 = []
odds_gg = []
odds_ng = []

for i in range(0, len(sublists_championships[5]), 18):
    # extract team names and odds for 1x2 market
    team1, team2, _, _, odds1, _, oddsX, _, odds2, *rest = sublists_championships[5][i:i+18]
    
    # check if 'GG' and 'NG' elements are present
    if 'GG' in rest:
        odds_gg.append(rest[rest.index('GG')+1])
    else:
        odds_gg.append(None)
    if 'NG' in rest:
        odds_ng.append(rest[rest.index('NG')+1])
    else:
        odds_ng.append(None)
    
    # append team names to respective lists
    teams1.append(team1)
    teams2.append(team2)

In [15]:
import re

championship_pattern = r'(?<=Popular First\n).*?(?=\n)'
teams_pattern = r'(?<=\n)[A-Za-z\s]+\n[A-Za-z\s]+\n'
time_pattern = r'\b(?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)[\s\d]{1,2}:[\d]{2}\b'
bet_categories_pattern = r'(?:1|X|2|O[\s\d\.]+|U[\s\d\.]+|GG|NG)(?=\n)'

# extract the information using regex
championship = re.search(championship_pattern, daily_coupon_text).group().strip()
teams = re.findall(teams_pattern, daily_coupon_text)
time = re.findall(time_pattern, daily_coupon_text)
bet_categories = re.findall(bet_categories_pattern, daily_coupon_text)

# extract the odds
odds_pattern = r'[\d]+\.[\d]+'
odds = []
for i in range(len(teams)):
    odds.append(re.findall(odds_pattern, daily_coupon_text.split(teams[i])[1])[0:7])

In [17]:
teams

['Football\nBasketball\nTennis\n',
 'Velez Sarsfield\nCentral Cordoba Santiago\n',
 'Benin\nRwanda\n',
 'Sierra Leone\nSao Tome and Principe\n',
 'Lewes\nCanvey Island\n',
 'Potters Bar Town FC\nBognor Regis Town\n',
 'Accrington\nPlymouth\n',
 'Barnsley\nSheffield Wed\n',
 'Banbury\nCurzon Ashton FC\n',
 'Farsley Celtic FC\nDarlington\n',
 'Kings Lynn Town\nAFC Fylde\n',
 'Southport\nAlfreton Town FC\n',
 'Spennymoor Town\nLeamington\n',
 'Gainsborough\nMarine\n',
 'Marske United FC\nLiversedge\n',
 'Alvechurch\nBarwell\n',
 'Stratford Town\nLeiston FC\n',
 'Bradford\nCarlisle\n',
 'Crawley Town\nDoncaster\n',
 'Gillingham\nCrewe\n',
 'Chesham United FC\nPlymouth Parkway\n',
 'Hanwell Town\nPoole Town\n',
 'Harrow Borough\nBracknell Town\n',
 'Weiden\nSV Donaustauf\n',
 'Markets are not available\nGermania Ratingen\n',
 'MSV Dusseldorf\nKray\n',
 'Oldenburger SV\nFC Dornbreite\n',
 'SC Union Nettetal\nTSV Meerbusch\n',
 'Sportfreunde Baumberg\nVfB Homberg\n',
 'SV Sonsbeck\nKleve\n',
